# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [10]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [11]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW2-bidding-3233000


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW2-binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [12]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [13]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [14]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
3C OK
PASS OK
4C ? NN-value: PASS 0.874
4H OK
PASS OK
PASS OK
PASS ? NN-value: 5C 0.731
analyzing the play
CA
C8 OK
C3 OK
C5 OK
S8 OK
ST OK
SQ OK
SA OK
DK OK
DT OK
D6 OK
D3 OK
D9 ? losing: 0.13
D7 OK
DA OK
D8 OK
DJ ? losing: 0.48
DQ OK
HJ OK
S7 OK


In [15]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '3C',
 'candidates': [{'call': '3C', 'insta_score': 0.977}],
 'samples': ['8x.8x.Tx.AKJ9xxx Kxxx.A9x.A8xxx.Q A9x.KJT.J9xx.T8x QJTx.Qxxxx.KQ.xx 1.00000',
  '8x.8x.Tx.AKJ9xxx Qx.Q9xx.Axxx.Qxx K9xx.JTxx.K98.8x AJTxx.AKx.QJxx.T 1.00000',
  '8x.8x.Tx.AKJ9xxx KJTxxx.AQJx.8x.x xx.K9x.KJxxx.T8x AQ9.Txxx.AQ9x.Qx 1.00000',
  '8x.8x.Tx.AKJ9xxx ATxx.QT9xx.Kxx.8 KQ9.Jxx.AJ98x.QT Jxxx.AKx.Qxx.xxx 1.00000',
  '8x.8x.Tx.AKJ9xxx KQT9xx.J9.K8xx.x x.Qxxxxx.A9x.T8x AJxx.AKT.QJxx.Qx 1.00000',
  '8x.8x.Tx.AKJ9xxx AT9xx.Qxx.KJ8x.x KQx.9x.9xxxx.T8x Jxx.AKJTxx.AQ.Qx 1.00000',
  '8x.8x.Tx.AKJ9xxx Jxx.AKJx.J8xxx.Q T9xx.Qxxx.Ax.T8x AKQx.T9x.KQ9x.xx 1.00000',
  '8x.8x.Tx.AKJ9xxx AQx.KJTx.AQJxx.x xx.Q9xxxx.8xx.Q8 KJT9xx.A.K9x.Txx 1.00000',
  '8x.8x.Tx.AKJ9xxx AKxx.KT.Q98x.QTx Qxx.QJxxx.KJxx.x JT9x.A9xx.Axx.8x 1.00000',
  '8x.8x.Tx.AKJ9xxx AQxx.QJxx.98xx.x Txxx.AKxx.AQJ.T8 KJ9.T9x.Kxxx.Qxx 1.00000',
  '8x.8x.Tx.AKJ9xxx AQ9x.xxx.KQxx.8x Jxxx.QJ9x.9xx.Tx KTx.AKTx.AJ8x.Qx 1.00000',
  '8x.8x.Tx.AKJ9xxx KQx.AKTxx

the engine very confidently opens `1C` and doesn't even consider `1N`

In [16]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

{'card': 'CA',
 'candidates': [{'card': 'CA',
   'insta_score': 0.9296,
   'expected_tricks_sd': 12.552,
   'p_make_contract': 1.0}],
 'samples': ['Qx.Qxx.AK9xxx.T8 T9xxxx.x.8x.Qxxx AKJ.AKJT9xx.QJx. 0.11406',
  'Txx.Q9.KQ98xxx.Q Q9xx.Txx.x.T8xxx AKJx.AKJxxx.AJx. 0.10460',
  'AK9x.Q9.QJ8xx.Qx QTxxxx.T.9x.T8xx J.AKJxxxxx.AKxx. 0.10152',
  'AKJ9x.KTxx.Jxx.Q Txx.x.9xxx.T8xxx Qxx.AQJ9xx.AKQ8. 0.10119',
  'Jxx.JTx.AKQJ8x.x QT9xx.x.xxx.T8xx AKx.AKQ9xxx.9x.Q 0.10037',
  'JTx.AKxx.K98xx.x Q9xxxx.T.xx.T8xx AK.QJ9xxx.AQJx.Q 0.09878',
  'Qxxx.x.AKJ9xx.Qx JT9xx.Qxx.8.T8xx AK.AKJT9xx.Qxxx. 0.09652',
  'Axx.AQTxx.Q9xx.Q QT9xx..J8xxx.T8x KJx.KJ9xxx.AK.xx 0.09599',
  'AKx.T9xxx.QJx.Q8 Q9xx..98xxx.Txxx JTxx.AKQJxx.AKx. 0.09466',
  'Kxx.QT9x.KQJxx.Q QJT9x.x.8xx.T8xx Axx.AKJxxx.A9x.x 0.09296'],
 'hcp': -1,
 'shape': -1}

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [17]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'D3',
 'candidates': [{'card': 'D3',
   'insta_score': 0.66,
   'expected_tricks_dd': 3.771,
   'expected_score_dd': -14},
  {'card': 'D8',
   'insta_score': 0.3387,
   'expected_tricks_dd': 3.771,
   'expected_score_dd': -14},
  {'card': 'DQ',
   'insta_score': 0.0,
   'expected_tricks_dd': 3.271,
   'expected_score_dd': -70}],
 'samples': ['8xxx.xx.T.AKJ9xx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx A.AJ8xxxx.K9x.xx 0.8388',
  '8x.Axx.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axx.J8xxxx.K9x.x 0.0030',
  '8xx.xxx.T.AKJ9xx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxx.K9x.xx 0.1157',
  '8xx.xx.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.AJ8xxxx.K9x.x 0.0000',
  '8.AJ8x.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axxx.xxxxx.K9x.x 0.4584',
  '8xx.A8x.T.AKJxxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.Jxxxxx.K9x.9x 0.1083',
  '8x.AJx.T.AKJ9xxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Axx.8xxxxx.K9x.x 0.0000',
  '8xx.Axx.T.AKJxxx JTxx.QT.AJxxxx.8 KQ9x.K9.Q8x.QTxx Ax.J8xxxx.K9x.9x 0.0042',
  '8xx.8x

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [18]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [19]:
display_lin(lin)

In [20]:
board = parse_lin(lin)

In [21]:
card_by_card = CardByCard(*board, models, sampler, False)

In [22]:
card_by_card.analyze()

analyzing the bidding
1C OK
2H OK
X ? NN-value: PASS 1.000
4H ? NN-value: 3H 0.663
4S ? NN-value: 5C 0.186
PASS OK
PASS OK
PASS OK
analyzing the play
DA
D3 OK
D5 OK
D6 OK
C8 OK
C3 OK
CJ OK
CA OK
S4 ? losing: 0.25
S5 OK
S8 OK
SJ OK
H5 ? losing: 0.18
HJ OK
HQ OK
HA OK
S2 ? losing: 0.15
SA OK
ST OK
S3 OK
H2 OK
SK OK
H4 OK
H3 OK
D8 OK
D2 OK
DJ OK
S6 OK
SQ OK
C6 OK
H7 OK
S7 OK
H8 OK
C7 OK
HK OK
S9 OK
C5 OK
C2 OK
CT OK
HT OK
CK OK
H9 OK
D7 OK
C4 OK
DQ OK
D4 OK
DK OK
H6 OK


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [23]:
card_by_card.cards['S4'].to_dict()

{'card': 'S4',
 'candidates': [{'card': 'HJ',
   'insta_score': 0.3251,
   'expected_tricks_dd': 7.86,
   'expected_score_dd': -53},
  {'card': 'D8',
   'insta_score': 0.0458,
   'expected_tricks_dd': 7.79,
   'expected_score_dd': -56},
  {'card': 'DQ',
   'insta_score': 0.1129,
   'expected_tricks_dd': 7.78,
   'expected_score_dd': -57},
  {'card': 'S4',
   'insta_score': 0.0608,
   'expected_tricks_dd': 7.61,
   'expected_score_dd': -65},
  {'card': 'ST',
   'insta_score': 0.0,
   'expected_tricks_dd': 7.6,
   'expected_score_dd': -66},
  {'card': 'CK',
   'insta_score': 0.1072,
   'expected_tricks_dd': 7.55,
   'expected_score_dd': -72},
  {'card': 'SK',
   'insta_score': 0.0899,
   'expected_tricks_dd': 7.18,
   'expected_score_dd': -87},
  {'card': 'C7',
   'insta_score': 0.2223,
   'expected_tricks_dd': 6.63,
   'expected_score_dd': -118},
  {'card': 'C6',
   'insta_score': 0.2223,
   'expected_tricks_dd': 6.63,
   'expected_score_dd': -118},
  {'card': 'C9',
   'insta_score': 0.

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [24]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'candidates': [{'card': 'HK',
   'insta_score': 0.776,
   'expected_tricks_sd': 11.043,
   'p_make_contract': 0.881},
  {'card': 'C8',
   'insta_score': 0.1343,
   'expected_tricks_sd': 11.214,
   'p_make_contract': 0.9}],
 'samples': ['KJxx.x.KQ8x.QJxx T9x.AJ8xxx.x.Txx AQ8x..J9xx.AK9xx 0.82481',
  'T8xx.A.QJ8xx.AQx AJx.8xxxx.9.JT9x KQ9x.J.Kxx.Kxxxx 0.81041',
  'J9xx.AJ.K8xx.A9x K8x.8xxxx.J.JTxx AQTx..Q9xx.KQxxx 0.80618',
  'AKJx.8.QJ8xx.xxx T8x.Jxxxx.x.JT9x Q9xx.A.K9x.AKQxx 0.80282',
  'AQTx..KJ8xxx.Txx 98.AJ8xxx.9x.QJx KJxxx.x.Q.AK9xxx 0.80133',
  'AJTx.8.Qxxx.AKJx 98x.Jxxxx.K9.Txx KQxx.A.J8x.Q9xxx 0.79019',
  'AT8x.x.QJxx.KQTx Kxx.A8xxx.98.9xx QJ9x.J.Kxx.AJxxx 0.78396',
  'KQ98.Ax.9xx.AQJ9 xx.J8xxx.J8xx.xx AJTxx..KQ.KTxxxx 0.78322',
  'A98x.A.QJxx.KTxx Kxx.J8xxx.8x.J9x QJTx.x.K9x.AQxxx 0.78308',
  'AK8x.xx.Qxxx.Kxx 9xx.AJ8xx.8.Qxxx QJTx..KJ9x.AJT9x 0.77473',
  'KJxx.Ax.QJxx.Qxx Axx.J8xxx.98.JT9 QT98..Kxx.AKxxxx 0.76972',
  'T8xx.A.KJ8xx.A9x KJ9.J8xxx.x.Qxxx AQxx.x.Q9